In [1]:
# check to see if FRED key is working

import os, sys, requests, pandas as pd
from datetime import datetime
from getpass import getpass
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Resolve your key safely for this runtime
FRED_API_KEY = os.getenv("FRED_API_KEY") or getpass("Paste your FRED API key (hidden): ").strip()
if not FRED_API_KEY:
    raise SystemExit("No FRED API key provided.")
os.environ["FRED_API_KEY"] = FRED_API_KEY

FRED_OBS_API = "https://api.stlouisfed.org/fred/series/observations"

def get_observations(series_id: str,
                     start: str = "2020-01-01",
                     end: str = datetime.today().strftime("%Y-%m-%d"),
                     timeout: int = 30) -> pd.DataFrame:
    """Fetch FRED observations; returns df indexed by date with a single <series_id> column."""
    params = {
        "series_id": series_id,
        "file_type": "json",
        "observation_start": start,
        "observation_end": end,
        "api_key": os.environ["FRED_API_KEY"],
    }
    r = requests.get(FRED_OBS_API, params=params, timeout=timeout)
    r.raise_for_status()
    payload = r.json()
    if "observations" not in payload:
        raise RuntimeError(f"No 'observations' in response for {series_id}: {payload}")
    df = pd.DataFrame(payload["observations"])
    if df.empty:
        raise RuntimeError(f"No rows for {series_id} in {start}..{end}")

    # Keep only the two fields we need, then normalize/rename
    df = df.loc[:, ["date", "value"]].copy()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df = df.dropna(subset=["date", "value"]).sort_values("date")
    df = df.set_index("date").rename(columns={"value": series_id})
    return df

def summarize(df: pd.DataFrame):
    sid = df.columns[0]
    print(f"[ok] {sid:<12} rows={len(df):>5}  last={df.index.max().date()}  value={df.iloc[-1,0]:.6f}")
    display(df.tail(5))

# Test monthly / weekly / daily series
series_to_test = ["CPIAUCSL", "IC4WSA", "DCOILBRENTEU"]
start = "2020-01-01"; end = datetime.today().strftime("%Y-%m-%d")

ok = True
for sid in series_to_test:
    try:
        df = get_observations(sid, start, end)
        summarize(df)
    except Exception as e:
        ok = False
        print(f"[fail] {sid}: {e}", file=sys.stderr)

print("\n✅ FRED key works." if ok else "\n⚠️ Some series failed — check key, dates, or IDs.")

[ok] CPIAUCSL     rows=   69  last=2025-09-01  value=324.368000


,CPIAUCSL
date,
2025-05-01,320.580
2025-06-01,321.500
2025-07-01,322.132
2025-08-01,323.364
2025-09-01,324.368


[ok] IC4WSA       rows=  299  last=2025-09-20  value=237500.000000


,IC4WSA
date,
2025-08-23,228500
2025-08-30,230750
2025-09-06,240750
2025-09-13,240250
2025-09-20,237500


[ok] DCOILBRENTEU rows= 1484  last=2025-11-11  value=63.860000


,DCOILBRENTEU
date,
2025-11-05,63.54
2025-11-06,63.41
2025-11-07,63.72
2025-11-10,63.01
2025-11-11,63.86



✅ FRED key works.
